# LightFM model for influence marketing
Using a LightFM model for recommending influencers to companies and brands can help identify influencers who are likely to have a positive impact on the target audience and improve the effectiveness of the influence marketing campaign. 


In this notebook, we will conduct the following steps:

1. Collect data about the influencers and their social media activity, and data about companies and brands.

2. Preprocess the data.

3. Define the problem: Relate and give a score to the relationships between companies and influencers.

4. Build the model. The model can be trained using the collected data and can then be used to make recommendations.

5. Evaluate the model.

6. Use the model for influence marketing. Once the model is trained and evaluated, use it to make personalized recommendations to companies and brands, suggesting a list of influencers who are a good fit for the brand and campaign.

In further steps, we will monitor and adjust the model (updating the data, retraining the model, or tweaking the parameters).

### Model theoretical explanation
In general, recommendation models can be divided into two categories: content-based and collaborative filtering. Content-based models recommend based on the similarity of items or users using their description or metadata. Collaborative filtering models compute the latent factors of users and items based on the assumption that people who express similar opinions on one item will have similar opinions on other items. The choice between the two models depends on data availability, with collaborative filtering being effective when sufficient ratings or feedbacks are available, and content-based models being useful when there is a lack of ratings and metadata is available. Content-based models are also useful for addressing cold-start issues.

To address cold-start issues, hybrid approaches have been proposed that combine content-based and collaborative filtering. One such approach is the hybrid matrix factorization model.

**LightFM** is a Python implementation of a hybrid recommendation algorithm for both implicit and explicit feedbacks. It is a content-collaborative model that represents users and items as linear combinations of their content features’ latent factors. User and item embeddings are estimated for every feature, and these features are added together to be the final representations for users and items. LightFM computes the representation for each item and user by retrieving the corresponding row or column in the feature matrix and adding together the embeddings for features with non-zero weights. The resulting representations produce scores for every item for a given user, with highly scored items being more likely to be interesting to the user.

#### Modelling approach
For further mathematical explanation, please go to https://github.com/microsoft/recommenders/blob/main/examples/02_model_hybrid/lightfm_deep_dive.ipynb.

### 1. Import Libraries

In [181]:
# Install all the libraries in requirements.txt
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scrapbook as sb
import requests
import io

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

System version: 3.8.7 (tags/v3.8.7:6503f05, Dec 21 2020, 17:59:51) [MSC v.1928 64 bit (AMD64)]
LightFM version: 1.16


### 2. Defining Variables

In [182]:
# default number of recommendations
K = 10
# percentage of data used for testing
TEST_PERCENTAGE = 0.25
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 20
# no of threads to fit model
NO_THREADS = 32
# regularisation for both user and item features
ITEM_ALPHA = 1e-6
USER_ALPHA = 1e-6

# seed for pseudonumber generations
SEED = 42

### 3. Retrieve Data

In [183]:
# Upload dataset with interacions
df = pd.read_csv('df_reco.csv')
df

,itemID,Account,Link,Followers,Audience Country,Authentic engagement,Category1,Hashtags,Cost Story,Cost Post,userID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co,rating
0,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', 'l...",468000.0,1092000.0,0,The Little Shop,Cars,"['instadaily', 'food', 'followback', 'instatra...",Brazil,195608,0.019495
1,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', 'l...",468000.0,1092000.0,1,The Cozy Kitchen,Sports with a ball,"['instalike', 'fashion', 'holidayseason', 'ski...",Egypt,415795,10.019495
2,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', 'l...",468000.0,1092000.0,2,The Roost,Art,"['blessings', 'foodlife', 'endurance', 'fitnes...",Turkey,116164,0.019495
3,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', 'l...",468000.0,1092000.0,3,The Wooden Spoon,Computers,"['foodislifee', 'familytime']",Iran,524384,0.019495
4,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', 'l...",468000.0,1092000.0,4,Sunflower Fields,Management,"['health', 'trendy']",Germany,307127,0.019495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96750,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,96,The Crusty Baguette,Food,"['instamood', 'education', 'beautifuldestinati...",China,74286,0.049290
96751,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,97,The Plaid Pail,Luxury,"['goodmorning', 'cricket', 'transportation', '...",Nigeria,108265,0.049290
96752,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,98,The Sugar House,Business,"['muscle', 'lifestyle', 'instatravel', 'instad...",Germany,82077,0.049290
96753,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,99,Urban Bites,Shows,"['cricket', 'lifeisgood']",Tanzania,445319,0.049290


As two of the variables are not in the desired format, we will transform them.

In [184]:
def Convert(string):
    li = list(string.split(","))
    return li

In [185]:
# Convert string columns 'Hastaghs' and 'Hashtags_co' to list.
h = []
h_co = []

for i in range(len(df['Hashtags'])):
    has = df['Hashtags'][i][1:-1]
    has_co = df['Hashtags'][i][1:-1]
    h.append(Convert(has))
    h_co.append(Convert(has_co))

df['Hashtags'] = h
df['Hashtags_co'] = h_co

In [186]:
df

,itemID,Account,Link,Followers,Audience Country,Authentic engagement,Category1,Hashtags,Cost Story,Cost Post,userID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co,rating
0,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",468000.0,1092000.0,0,The Little Shop,Cars,"['football', 'entrepreneur', 'foodstylist', ...",Brazil,195608,0.019495
1,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",468000.0,1092000.0,1,The Cozy Kitchen,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",Egypt,415795,10.019495
2,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",468000.0,1092000.0,2,The Roost,Art,"['football', 'entrepreneur', 'foodstylist', ...",Turkey,116164,0.019495
3,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",468000.0,1092000.0,3,The Wooden Spoon,Computers,"['football', 'entrepreneur', 'foodstylist', ...",Iran,524384,0.019495
4,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"['football', 'entrepreneur', 'foodstylist', ...",468000.0,1092000.0,4,Sunflower Fields,Management,"['football', 'entrepreneur', 'foodstylist', ...",Germany,307127,0.019495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96750,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,96,The Crusty Baguette,Food,['photo'],China,74286,0.049290
96751,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,97,The Plaid Pail,Luxury,['photo'],Nigeria,108265,0.049290
96752,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,98,The Sugar House,Business,['photo'],Germany,82077,0.049290
96753,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,['photo'],18336.0,42784.0,99,Urban Bites,Shows,['photo'],Tanzania,445319,0.049290


### 4. Extract and prepare item features.

The influencers' hashtags will be used as the item metadata.

In [187]:
all_hashtags = []
for x in df['Hashtags']:
  all_hashtags.append(x)

all_hashtags = sorted(list(set(itertools.chain.from_iterable(all_hashtags))))
all_hashtags

[" 'amazing'",
 " 'art'",
 " 'athlete'",
 " 'baby'",
 " 'baseball'",
 " 'basketball'",
 " 'beach'",
 " 'beautiful'",
 " 'beautifulday'",
 " 'beautifuldestinations'",
 " 'beauty'",
 " 'beautyblogger'",
 " 'beautyful'",
 " 'bestoftheday'",
 " 'blackandwhite'",
 " 'blessed'",
 " 'blessings'",
 " 'blogger'",
 " 'boxing'",
 " 'cardio'",
 " 'cars'",
 " 'coffee'",
 " 'colorful'",
 " 'construction'",
 " 'cool'",
 " 'couplesgoals'",
 " 'cricket'",
 " 'crossfit'",
 " 'cute'",
 " 'cycling'",
 " 'dog'",
 " 'doglover'",
 " 'doglovers'",
 " 'dogsofinstagram'",
 " 'dogstagram'",
 " 'education'",
 " 'endurance'",
 " 'energy'",
 " 'entertainment'",
 " 'entrepreneur'",
 " 'extremesports'",
 " 'family'",
 " 'familytime'",
 " 'fashion'",
 " 'fashionable'",
 " 'fashionblogger'",
 " 'fashionista'",
 " 'fashionphotography'",
 " 'fashionstyle'",
 " 'fit'",
 " 'fitfam'",
 " 'fitlife'",
 " 'fitness'",
 " 'fitnessmodel'",
 " 'fitnessmotivation'",
 " 'fitspo'",
 " 'flex'",
 " 'follow'",
 " 'follow4follow'",
 " 'f

In [188]:
all_categories = sorted(list(set(df['Category1'])))
all_categories

['Accessories',
 'Adult content',
 'Animals',
 'Art',
 'Beauty',
 'Business',
 'Cars',
 'Cinema',
 'Clothing',
 'Computers',
 'Extreme',
 'Family',
 'Fashion',
 'Finance',
 'Fitness',
 'Food',
 'Humor',
 'Kids',
 'Lifestyle',
 'Literature',
 'Luxury',
 'Machinery',
 'Management',
 'Modeling',
 'Music',
 'Nature  landscapes',
 'Photography',
 'Racing',
 'Science',
 'Shows',
 'Sports with a ball',
 'Undefined']

In [189]:
all_countries = sorted(list(set(df['Audience Country'])))
all_countries

['Algeria',
 'Argentina',
 'Brazil',
 'Chile',
 'China',
 'Colombia',
 'Egypt',
 'France',
 'Germany',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Italy',
 'Japan',
 'Kazakhstan',
 'Mexico',
 'Morocco',
 'Nigeria',
 'Philippines',
 'Poland',
 'Russia',
 'Saudi Arabia',
 'South Korea',
 'Spain',
 'Syria',
 'Thailand',
 'Turkey',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Unknown']

### 5. Extract and prepare user features.

In [190]:
all_hashtagsCo = []
for x in df['Hashtags_co']:
  all_hashtagsCo.append(x)

all_hashtagsCo = sorted(list(set(itertools.chain.from_iterable(all_hashtagsCo))))
all_hashtagsCo

[" 'amazing'",
 " 'art'",
 " 'athlete'",
 " 'baby'",
 " 'baseball'",
 " 'basketball'",
 " 'beach'",
 " 'beautiful'",
 " 'beautifulday'",
 " 'beautifuldestinations'",
 " 'beauty'",
 " 'beautyblogger'",
 " 'beautyful'",
 " 'bestoftheday'",
 " 'blackandwhite'",
 " 'blessed'",
 " 'blessings'",
 " 'blogger'",
 " 'boxing'",
 " 'cardio'",
 " 'cars'",
 " 'coffee'",
 " 'colorful'",
 " 'construction'",
 " 'cool'",
 " 'couplesgoals'",
 " 'cricket'",
 " 'crossfit'",
 " 'cute'",
 " 'cycling'",
 " 'dog'",
 " 'doglover'",
 " 'doglovers'",
 " 'dogsofinstagram'",
 " 'dogstagram'",
 " 'education'",
 " 'endurance'",
 " 'energy'",
 " 'entertainment'",
 " 'entrepreneur'",
 " 'extremesports'",
 " 'family'",
 " 'familytime'",
 " 'fashion'",
 " 'fashionable'",
 " 'fashionblogger'",
 " 'fashionista'",
 " 'fashionphotography'",
 " 'fashionstyle'",
 " 'fit'",
 " 'fitfam'",
 " 'fitlife'",
 " 'fitness'",
 " 'fitnessmodel'",
 " 'fitnessmotivation'",
 " 'fitspo'",
 " 'flex'",
 " 'follow'",
 " 'follow4follow'",
 " 'f

The companies' categories will be used as the user metadata.

In [191]:
all_categoriesCo = sorted(list(set(df['Category_co'])))
all_categoriesCo

['Accessories',
 'Adult content',
 'Art',
 'Beauty',
 'Business',
 'Cars',
 'Cinema',
 'Clothing',
 'Computers',
 'Family',
 'Fashion',
 'Finance',
 'Fitness',
 'Food',
 'Humor',
 'Kids',
 'Lifestyle',
 'Luxury',
 'Machinery',
 'Management',
 'Modeling',
 'Music',
 'Nature  landscapes',
 'Photography',
 'Racing',
 'Science',
 'Shows',
 'Sports with a ball',
 'Undefined']

In [192]:
all_countriesCo = sorted(list(set(df['Country_co'])))
all_countriesCo

['Bangladesh',
 'Brazil',
 'China',
 'Colombia',
 'DR Congo',
 'Egypt',
 'Ethiopia',
 'France',
 'Germany',
 'India',
 'Indonesia',
 'Iran',
 'Italy',
 'Japan',
 'Kenya',
 'Mexico',
 'Myanmar',
 'Nigeria',
 'Pakistan',
 'Philippines',
 'Russia',
 'South Africa',
 'South Korea',
 'Spain',
 'Tanzania',
 'Thailand',
 'Turkey',
 'United Kingdom',
 'United States',
 'Vietnam']

Before fitting the LightFM model, we need to create an instance of Dataset which holds the interaction matrix.

The data is required to be converted into a Dataset instance and then create a user/item id mapping with the fit method.

In [193]:
dataset2 = Dataset()
dataset2.fit(df['userID'], 
            df['itemID'], 
            item_features=all_hashtags,
            user_features=all_categoriesCo)

The hashtags are then converted into a item feature matrix using the build_item_features method as follows:

In [194]:
item_features = dataset2.build_item_features((x, y) for x,y in zip(df.itemID, df.Hashtags))

The user occupations are then converted into an user feature matrix using the build_user_features method as follows:

In [195]:
user_features = dataset2.build_user_features((x, [y]) for x,y in zip(df.userID, df['Category_co']))

We will use cross_validation.random_train_test_split method to split the interaction data and splits it into two disjoint training and test sets.

Once the item and user features matrices have been completed, we build the interaction matrix and split the interactions into train and test sets as follows:

In [196]:
interactions2, weights2 = dataset2.build_interactions(df.iloc[:, [10,0,16]].values)

train_interactions2, test_interactions2 = cross_validation.random_train_test_split(
    interactions2, 
    test_percentage=TEST_PERCENTAGE,
    random_state=np.random.RandomState(SEED)
)

### 6. Fit the LightFM model with additional user and item features

The LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. It maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present, as in our case.

In [197]:
model2 = LightFM(loss='warp', no_components=NO_COMPONENTS, 
                 learning_rate=LEARNING_RATE, 
                 item_alpha=ITEM_ALPHA,
                 user_alpha=USER_ALPHA,
                 random_state=np.random.RandomState(SEED)
                )

In [198]:
model2.fit(interactions=train_interactions2,
           user_features=user_features,
           item_features=item_features,
           epochs=NO_EPOCHS
           )

### 7. Prepare model evaluation data

The evaluation data needs to be prepared in order to get them into a format consumable with this repo's evaluation methods. Firstly the train/test indices and id mappings are extracted using the new interations matrix as follows:

In [199]:
uids, iids, interaction_data = cross_validation._shuffle(
    interactions2.row, 
    interactions2.col, 
    interactions2.data, 
    random_state=np.random.RandomState(SEED)
)

uid_map, ufeature_map, iid_map, ifeature_map = dataset2.mapping()
cutoff = int((1.0 - TEST_PERCENTAGE) * len(uids))
test_idx = slice(cutoff, None)

The test dataframe is then constructed as follows:

In [200]:
with Timer() as test_time:
    test_df2 = prepare_test_df(test_idx, uids, iids, uid_map, iid_map, weights2)
print(f"Took {test_time.interval:.1f} seconds for prepare and predict test data.") 

Took 2.9 seconds for prepare and predict test data.


The predictions of all unseen user-item pairs can be prepared as follows:

In [201]:
with Timer() as test_time:
    all_predictions2 = prepare_all_predictions(df, uid_map, iid_map, 
                                              interactions=train_interactions2,
                                               user_features=user_features,
                                               item_features=item_features,
                                               model=model2,
                                               num_threads=NO_THREADS)

print(f"Took {test_time.interval:.1f} seconds for prepare and predict all data.")

Took 13.1 seconds for prepare and predict all data.


In [202]:
all_predictions2[all_predictions2['userID']==38].sort_values(by=['prediction'], ascending=False)

,userID,itemID,prediction
10371,38,976,-103.546593
10327,38,819,-103.691322
10186,38,246,-103.714485
10225,38,438,-103.842407
10236,38,481,-103.954834
...,...,...,...
10216,38,400,-106.989761
10234,38,478,-107.254051
10348,38,881,-107.272110
10382,38,1001,-107.304924


In [203]:
df[(df['itemID'] == 342) & (df['userID'] == 38)]

,itemID,Account,Link,Followers,Audience Country,Authentic engagement,Category1,Hashtags,Cost Story,Cost Post,userID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co,rating
33003,342,camilo,https://www.instagram.com/camilo/,25800000,Argentina,246100.0,Music,"['fashion', 'manufacturing', 'snowboarding']",14766.0,34454.0,38,Garden Grille,Sports with a ball,"['fashion', 'manufacturing', 'snowboarding']",Japan,552039,0.009539


### 8. Model evaluation and comparison

The predictive performance of the new model can be computed and compared with the previous model (which used only the explicit rating) as follows:

In [204]:
eval_precision2 = precision_at_k(rating_true=test_df2, 
                                rating_pred=all_predictions2, k=K)
eval_recall2 = recall_at_k(test_df2, all_predictions2, k=K)

print(
    "------ Using Repo's evaluation methods ------",
    f"Precision@K:\t{eval_precision2:.6f}",
    f"Recall@K:\t{eval_recall2:.6f}")

------ Using Repo's evaluation methods ------ Precision@K:	0.889000 Recall@K:	0.036823


### 9. Similar users and items

As the LightFM package operates based on latent embeddings, these can be retrieved once the model has been fitted to assess user-user and/or item-item affinity.

#### User affinity

The user-user affinity can be retrieved with the get_user_representations method from the fitted model as follows:

In [205]:
_, user_embeddings = model2.get_user_representations(features=user_features)
user_embeddings

array([[-0.31070885,  0.464963  , -0.20998643, ..., -0.05488526,
         0.10889013, -0.6950951 ],
       [ 0.20619848,  0.6584211 ,  0.64566934, ...,  0.74537873,
         0.0619914 ,  0.6915591 ],
       [ 0.23044176, -0.1083559 ,  0.43992823, ..., -1.0249213 ,
        -0.7629782 ,  0.2976227 ],
       ...,
       [ 0.2307572 , -0.10767306,  0.43954405, ..., -1.0247136 ,
        -0.76257974,  0.29799822],
       [-0.5170609 ,  0.20785686, -0.7845854 , ...,  1.030279  ,
        -1.0604037 ,  0.19365366],
       [-0.24879871,  0.32858348, -1.3619444 , ..., -0.00229555,
        -1.234457  , -0.3985371 ]], dtype=float32)

In order to retrieve the top N similar users, we can use the similar_users from recommenders. For example, if we want to choose top 10 users most similar to the user 1:

In [206]:
similar_users(user_id=100, 
              user_features=user_features, 
              model=model2)

,userID,score
0,99,0.582721
1,81,0.582690
2,94,0.582556
3,60,0.290299
4,23,0.290218
5,18,0.250063
6,44,0.249698
7,40,0.249581
8,91,0.249516
9,78,0.249472


#### Item affinity

The item-item affinity can be retrieved with the get_item_representations method using the fitted model.

In [207]:
similar_items(item_id=10, 
              item_features=item_features, 
              model=model2)

,itemID,score
0,738,0.686143
1,805,0.641785
2,683,0.620605
3,946,0.608882
4,236,0.587532
5,173,0.572627
6,699,0.568218
7,449,0.557170
8,681,0.553894
9,499,0.545237


The predict() method of the model object takes a user ID and item ID as input and returns a predicted score for the user-item pair. To get the predicted scores for all items in the dataset for the specified user, we pass the user ID and an array of item IDs from 0 to num_items-1 to the predict() method. We then use np.argsort() to rank the items in descending order by predicted score, and take the top N items from the sorted list.

### 10. Recommendations

Demo Recommendations for a specific user (company), returning 10 top recommended items (influencers).

In [208]:
user_id = 1

# Get the predicted scores for all items
scores = model2.predict(user_id, np.arange(1000))
top_items = np.argsort(-scores)

# Print the top N items
N = 10
print("Top {} recommended items for user {}: {}".format(N, user_id, top_items[:N]))

Top 10 recommended items for user 1: [700 127 146 264 691 343 625 477 965 881]
